In [1]:
from gensim.corpora import Dictionary
import tqdm
import sys
from gensim import corpora
from gensim import corpora, models, similarities
import numpy as np
from gensim.summarization.bm25 import get_bm25_weights
from gensim.summarization.bm25 import BM25
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
dct_title = Dictionary(prune_at=None)
# dct_body = Dictionary(prune_at=None)

In [ ]:
with open ("../data/docs.tsv") as fin:
    for doc in tqdm.tqdm_notebook (fin):
        doc = doc.strip ().split("\t")[1:]
        
        if len (doc) > 0:
            dct_title.add_documents ([doc[0].split (" ")], prune_at=None)
        if len (doc) > 1:
            dct_body.add_documents ([doc[1].split (" ")], prune_at=None)

In [ ]:
dct_title.save ("../result/dct_title.dump")
dct_body.save ("../result/dct_body.dump")

In [ ]:
dct_title = dct_title.load ("../result/dct_title.dump")

In [ ]:
def doc_reader (title):
    doc_num = 0 if title else 1
    
    dct = Dictionary(prune_at=None)
    dct = dct.load ("../result/dct_{}.dump".format ("title" if title else "body"))
    
    with open ("../data/docs.tsv") as fin:
        for i, doc in enumerate (fin):
            doc = doc.strip ().split("\t")[1:]
            if len (doc) > doc_num:
                yield dct.doc2bow(doc[doc_num].split (" "))
            else:
                yield []

In [ ]:
corpora.MmCorpus.serialize('../result/corpus_title_only.mm', doc_reader (title=True))

In [ ]:
corpus = corpora.MmCorpus('../result/corpus_title_only.mm')

In [ ]:
bm25 = BM25 (corpus)

In [ ]:
queries = {}
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").upper ().split ("\t")
        queries[int (q[0])] = dct_title.doc2bow (q[1].split ())

In [ ]:
with open ("../result/result_bm25_aver_0_try_0.csv", "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for key in tqdm.tqdm (sorted (queries.iterkeys ())):
        for doc in np.argsort (bm25.get_scores(queries[key], 0))[-5:][::-1]:
            fout.write ("{},{}\n".format (key, doc))




# body




In [ ]:
corpora.MmCorpus.serialize('../result/corpus_body_only.mm', doc_reader (title=False))

In [ ]:
corpus = corpora.MmCorpus('../result/corpus_body_only.mm')

In [ ]:
for doc in corpus:
    for rowd in doc:
        print rowd
    break

In [ ]:
bm25 = BM25 (corpus)

In [ ]:
def reqd_queries (title):
    queries = {}
    with open ("../data/queries.tsv") as fin:
        dct = dct_title.load ("../result/dct_{}.dump".format ("title" if title else "body"))
        for q in fin:
            q = q.strip ().decode ("utf-8").upper ().split ("\t")
            queries[int (q[0])] = dct.doc2bow (q[1].split ())
        return queries

In [ ]:
with open ("../result/result_bm25_aver_0_try_0_body.csv", "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for key in tqdm.tqdm (sorted (reqd_queries(False).iterkeys ())):
        for doc in np.argsort (bm25.get_scores(queries[key], 0))[-5:][::-1]:
            fout.write ("{},{}\n".format (key, doc))

# bm25

In [ ]:
import math

In [ ]:
PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25


class BM25(object):
    """Implementation of Best Matching 25 ranking function.

    Attributes
    ----------
    corpus_size : int
        Size of corpus (number of documents).
    avgdl : float
        Average length of document in `corpus`.
    corpus : list of list of str
        Corpus of documents.
    f : list of dicts of int
        Dictionary with terms frequencies for each document in `corpus`. Words used as keys and frequencies as values.
    df : dict
        Dictionary with terms frequencies for whole `corpus`. Words used as keys and frequencies as values.
    idf : dict
        Dictionary with inversed terms frequencies for whole `corpus`. Words used as keys and frequencies as values.
    doc_len : list of int
        List of document lengths.
    """

    def __init__(self, corpus):
        """
        Parameters
        ----------
        corpus : list of list of str
            Given corpus.

        """
        self.corpus_size = len(corpus)
        self.corpus = corpus
        self.df = {}
        self.idf = {}
        self.doc_len = []
        self.initialize()

    def initialize(self):
        """Calculates frequencies of terms in documents and in corpus. Also computes inverse document frequencies."""
        len___my = 0
        for document in tqdm.tqdm (self.corpus):
            document = dict (document)
            
            doc_len = sum (document.itervalues ())
            self.doc_len.append(doc_len)

            for word, freq in document.iteritems ():
                if word not in self.df:
                    self.df[word] = 0
                self.df[word] += 1
    

        self.avgdl = float (sum (self.doc_len)) / self.corpus_size
        
        for word, freq in self.df.iteritems ():
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            

    def get_score(self, document, corp_doc, index, average_idf):
        """Computes BM25 score of given `document` in relation to item of corpus selected by `index`.

        Parameters
        ----------
        document : list of str
            Document to be scored.
        index : int
            Index of document in corpus selected to score with `document`.
        average_idf : float
            Average idf in corpus.

        Returns
        -------
        float
            BM25 score.

        """
        score = 0
        for key in set (document.keys ()) & set (corp_doc.keys ()):
            idf = self.idf[key] if self.idf[key] >= 0 else EPSILON * average_idf
            score += (idf * corp_doc[key] * (PARAM_K1 + 1)
                      / (corp_doc[key] + PARAM_K1 * (1 - PARAM_B + PARAM_B * self.doc_len[index] / self.avgdl)))
        return score

    def get_scores(self, document, average_idf):
        """Computes and returns BM25 scores of given `document` in relation to
        every item in corpus.

        Parameters
        ----------
        document : list of str
            Document to be scored.
        average_idf : float
            Average idf in corpus.

        Returns
        -------
        list of float
            BM25 scores.

        """
        document = dict(document)
        scores = []
        for doc_num, corp_doc in enumerate (self.corpus):
            score = self.get_score(document, dict (corp_doc), doc_num, average_idf)
            scores.append(score)
        return scores


In [ ]:
bm25 = BM25 (corpus)

In [ ]:
with open ("../result/result_bm25_aver_0_try_0_title_self.csv", "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for key in tqdm.tqdm (sorted (queries.iterkeys ())):
        for doc in np.argsort (bm25.get_scores(queries[key], 0))[-5:][::-1]:
            fout.write ("{},{}\n".format (key, doc))

In [ ]:
queries = reqd_queries (True)

In [ ]:
def reqd_queries (title):
    queries = {}
    with open ("../data/queries.tsv") as fin:
        dct = dct_title.load ("../result/dct_{}.dump".format ("title" if title else "body"))
        for q in fin:
            q = q.strip ().decode ("utf-8").upper ().split ("\t")
            queries[int (q[0])] = dct.doc2bow (q[1].split ())
        return queries

# bmtry

In [5]:
def doc_reader (title):
    doc_num = 0 if title else 1
    
    with open ("../data/docs.tsv") as fin:
        for i, doc in tqdm.tqdm (enumerate (fin)):
            doc = doc.strip ().split("\t")[1:]
            if len (doc) > doc_num:
                yield doc[doc_num].split (" ")
            else:
                yield []

In [6]:
bm25 = BM25 (list (doc_reader (True)))

582167it [10:10, 953.08it/s]


In [7]:
def read_queries (title):
    queries = {}
    with open ("../data/queries.tsv") as fin:
        
        for q in fin:
            q = q.strip ().decode ("utf-8").upper ().split ("\t")
            queries[int (q[0])] = q[1].split ()
        return queries

In [9]:
queries = read_queries (True)

In [10]:
with open ("../result/result_bm25_aver_0_try_0_title_self.csv", "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for key in tqdm.tqdm (sorted (queries.iterkeys ())):
        for doc in np.argsort (bm25.get_scores(queries[key], 0))[-5:][::-1]:
            fout.write ("{},{}\n".format (key, doc))

100%|██████████| 6311/6311 [46:58<00:00,  2.24it/s]
